In [81]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm_notebook
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns



plt.style.use('seaborn')
sns.set(font_scale=1)

import gc

In [82]:
train = pd.read_csv("../input/landslide-prevention-and-innovation-challenge/Train (12).csv")
test = pd.read_csv("../input/landslide-prevention-and-innovation-challenge/Test (13).csv")
sample = pd.read_csv('../input/landslide-prevention-and-innovation-challenge/SampleSubmission (7).csv')

In [83]:
train

,Sample_ID,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,...,17_sdoif,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif,Label
0,1,130,129,127,126,123,126,125,124,122,...,1.281779,1.281743,1.281720,1.281684,1.281811,1.281788,1.281752,1.281729,1.281693,0
1,2,161,158,155,153,151,162,159,155,153,...,1.359639,1.359608,1.359587,1.359556,1.359683,1.359662,1.359631,1.359610,1.359579,1
2,3,149,151,154,156,158,154,157,158,160,...,1.365005,1.365025,1.365055,1.365075,1.364937,1.364967,1.364988,1.365018,1.365038,0
3,4,80,78,77,75,73,80,78,77,75,...,1.100708,1.100738,1.100759,1.100789,1.100630,1.100650,1.100680,1.100700,1.100731,0
4,5,117,115,114,112,110,115,113,111,110,...,1.284180,1.284130,1.284056,1.284006,1.284125,1.284050,1.284001,1.283926,1.283876,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,10860,203,202,201,200,199,202,201,201,199,...,1.271912,1.272011,1.272076,1.272174,1.271720,1.271786,1.271884,1.271950,1.272048,0
10860,10861,432,430,427,423,421,431,429,426,422,...,1.331153,1.331137,1.331113,1.331096,1.331207,1.331183,1.331167,1.331142,1.331126,1
10861,10862,348,345,342,339,335,344,342,339,336,...,1.298362,1.298337,1.298321,1.298297,1.298392,1.298376,1.298352,1.298336,1.298311,1
10862,10863,93,91,90,88,86,96,94,93,92,...,1.355035,1.355061,1.355099,1.355125,1.354937,1.354976,1.355002,1.355041,1.355067,0


In [129]:
train.columns

Index(['1_elevation', '2_elevation', '3_elevation', '4_elevation',
       '5_elevation', '6_elevation', '7_elevation', '8_elevation',
       '9_elevation', '10_elevation',
       ...
       '17_sdoif', '18_sdoif', '19_sdoif', '20_sdoif', '21_sdoif', '22_sdoif',
       '23_sdoif', '24_sdoif', '25_sdoif', 'Label'],
      dtype='object', length=226)

In [84]:
id_=test["Sample_ID"]
test.drop("Sample_ID", axis=1, inplace=True)
train.drop("Sample_ID", axis=1, inplace=True)

In [85]:
y_train = train[['Label']]
df_train=train.drop(['Label'], axis=1)

In [86]:
y_train.nunique()

Label    2
dtype: int64

In [87]:
cols=df_train.columns

In [88]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=5168)
# oof = df_train[['ID_code', 'target']]
# oof['predict'] = 0
# predictions = test[['ID_code']]
# feature_importance_df = pd.DataFrame()
# val_aucs = []
for fold, (trn_idx, val_idx) in enumerate(skf.split(df_train, y_train)):
    X_train, y_train = df_train.iloc[trn_idx][cols], train.iloc[trn_idx]['Label']
    X_valid, y_valid = df_train.iloc[val_idx][cols], train.iloc[val_idx]['Label']
    break
    
    
    
clf = CatBoostClassifier(loss_function = "CrossEntropy", eval_metric = "AUC",random_seed=123,use_best_model=True,
                          learning_rate=0.01,  iterations=8876 ,verbose=100,
                           bootstrap_type= "Poisson", 
                           task_type="GPU", 
#                              l2_leaf_reg= 16.5056753964314982, depth= 3.0,
#                              fold_len_multiplier= 2.9772639036842174, 
#                              scale_pos_weight= 3.542962442406767, 
#                              fold_permutation_block_size=16.0, subsample= 0.46893530376570957
#                              fold_len_multiplier=3.2685541035861747, 
#                              scale_pos_weight= 2.6496926337120916, 
#                              fold_permutation_block_size= 6.0, 
                          )
print("Model training")
clf.fit(X_train, y_train,  eval_set=(X_valid, y_valid), early_stopping_rounds=2000,verbose=100)

Model training


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7781893	best: 0.7781893 (0)	total: 46.9ms	remaining: 7m 49s
100:	test: 0.8276480	best: 0.8276480 (100)	total: 4.84s	remaining: 7m 54s
200:	test: 0.8391194	best: 0.8391194 (200)	total: 9.18s	remaining: 7m 27s
300:	test: 0.8474603	best: 0.8474603 (300)	total: 13.4s	remaining: 7m 12s
400:	test: 0.8533607	best: 0.8533788 (398)	total: 17.8s	remaining: 7m 5s
500:	test: 0.8573169	best: 0.8574161 (498)	total: 22s	remaining: 6m 56s
600:	test: 0.8608716	best: 0.8608896 (599)	total: 26.1s	remaining: 6m 48s
700:	test: 0.8637631	best: 0.8637947 (697)	total: 30.4s	remaining: 6m 43s
800:	test: 0.8665058	best: 0.8665058 (800)	total: 34.7s	remaining: 6m 38s
900:	test: 0.8686305	best: 0.8686305 (900)	total: 39.7s	remaining: 6m 40s
1000:	test: 0.8703176	best: 0.8703176 (1000)	total: 43.8s	remaining: 6m 34s
1100:	test: 0.8716664	best: 0.8716664 (1100)	total: 48s	remaining: 6m 27s
1200:	test: 0.8731911	best: 0.8732182 (1199)	total: 52.3s	remaining: 6m 22s
1300:	test: 0.8742512	best: 0.8742602 (1

In [93]:
cols=test.columns
predict = clf.predict_proba(test[cols])

In [94]:
predict

array([[0.73226669, 0.26773331],
       [0.99336331, 0.00663669],
       [0.99888529, 0.00111471],
       ...,
       [0.98638225, 0.01361775],
       [0.98871418, 0.01128582],
       [0.51011293, 0.48988707]])

In [119]:
l=[]
for i in predict:
    l.append(np.argmax(i))

In [120]:
sub=pd.DataFrame({"Sample_ID":id_,"Label":l})

In [122]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")
create_submission(sub, "sol")

/kaggle/working/sol.csv